In [28]:
import pandas as pd
import numpy as np

In [2]:
path = '/Users/pranav/nfl_betting_system/data_processing_files/advanced_game_info.csv'
df = pd.read_csv(path)

In [3]:
def normalize(df):
    df = df.replace({True:1, False:0})
    return df
df = normalize(df)

In [4]:
def updateTable(df):
    for idx in range(len(df)):
        if df.loc[idx, 'week'] == 1:
            temp = pd.read_csv('/Users/pranav/nfl_betting_system/team_records/' + str(int(df.loc[idx, 'season'])-1) + '.csv')
            if df.loc[idx, 'team_home'] in set(temp['Tm']):
                df.loc[idx, 'home_win%'] = float(temp.loc[temp['Tm'] == df.loc[idx, 'team_home']]['win%'])
            if df.loc[idx, 'team_away'] in set(temp['Tm']):
                df.loc[idx, 'away_win%'] = float(temp.loc[temp['Tm'] == df.loc[idx, 'team_away']]['win%'])
    return df

In [5]:
df = updateTable(df)

In [6]:
def pythagorean_win_EV(df):
    arr = np.empty((0,2))
    
    for idx, row in df.iterrows():
        
        home_pf = row.home_team_ppg * row.home_team_GP
        home_pa = row.home_team_oppg * row.home_team_GP
        away_pf = row.away_team_ppg * row.away_team_GP
        away_pa = row.away_team_oppg * row.away_team_GP
        
        
        home_exp = (1.5) * np.log(row.home_team_ppg+row.home_team_oppg)
        away_exp = (1.5) * np.log(row.away_team_ppg+row.away_team_oppg)
        
        
        home_py = np.round((np.power(home_pf, home_exp)/(np.power(home_pf, home_exp) + np.power(home_pa, home_exp))), 
                           3)
        away_py = np.round((np.power(away_pf, away_exp)/(np.power(away_pf, away_exp) + np.power(away_pa, away_exp))), 
                           3)
        
        arr = np.append(arr, values = [[home_py, away_py]], axis = 0)
        
    return arr



def addEVs(df):
    temp = pd.DataFrame(pythagorean_win_EV(df), columns=['home_win_exp', 'away_win_exp'])
    
    df = pd.concat([df, temp], axis = 1)
    
    return df

In [7]:
#our pythagorean calculation will result in quite a few 'divide by 0' errors and 'log(0)' ones as well 
#the following lines will ignore these messages
import warnings
warnings.simplefilter('ignore')
df = addEVs(df)

In [8]:
to_drop = ['season', 'week', 'team_home', 'team_away', 'team_favorite_id', 
           'home_team_GP', 'home_team_wins', 'away_team_GP', 'away_team_wins']

df.drop(to_drop, axis=1, inplace=True)
df.dropna(axis=0, inplace=True)
df.index = np.array(range(len(df)))

In [9]:
df.drop(['home_team_id', 'away_team_id', 'winner'], 
        axis=1, 
        inplace=True)

In [11]:
df.sample(15)

,score_home,score_away,spread_favorite,point_diff,spread_covered,over_under_line,total_points,over_under_result,home_team_fav,home_win%,home_team_ppg,home_team_oppg,home_p_diff,away_win%,away_team_ppg,away_team_oppg,away_p_diff,home_win_exp,away_win_exp
4583,10,16,-5.0,6,0,47.5,26,0,1,0.667,22.67,20.67,2.00,0.333,26.33,23.33,3.00,0.628,0.670
3158,23,26,-2.5,3,0,43.0,49,1,1,0.750,22.75,22.75,0.00,0.500,22.75,21.75,1.00,0.500,0.564
773,38,17,-8.5,21,1,39.5,55,1,1,0.600,23.80,23.20,0.60,0.500,17.25,28.25,-11.00,0.537,0.056
1399,24,10,-5.5,14,1,43.0,34,0,1,0.500,19.00,20.07,-1.07,0.286,19.71,25.50,-5.79,0.425,0.187
2159,23,27,-3.0,4,1,41.0,50,1,0,0.000,7.00,19.00,-12.00,1.000,14.00,12.00,2.00,0.008,0.680
4286,19,10,-10.0,9,0,46.0,29,0,1,0.857,31.29,19.93,11.36,0.429,20.07,23.14,-3.07,0.935,0.309
3431,28,26,-7.0,2,0,43.5,54,1,0,0.200,20.20,30.00,-9.80,0.600,26.60,22.60,4.00,0.089,0.722
2201,10,38,-10.5,28,1,41.0,48,1,0,0.000,6.00,27.00,-21.00,1.000,29.50,20.00,9.50,0.000,0.907
3841,33,18,-5.0,15,0,48.0,51,1,0,0.000,19.00,25.00,-6.00,0.500,30.50,15.00,15.50,0.174,0.983
2859,45,17,-7.0,28,1,55.0,62,1,1,0.800,33.47,21.47,12.00,0.400,25.93,25.60,0.33,0.935,0.519


In [13]:
df['sq_err'] = (df['total_points'] - df['over_under_line'])**2

In [30]:
df.sample(100)

,score_home,score_away,spread_favorite,point_diff,spread_covered,over_under_line,total_points,over_under_result,home_team_fav,home_win%,home_team_ppg,home_team_oppg,home_p_diff,away_win%,away_team_ppg,away_team_oppg,away_p_diff,home_win_exp,away_win_exp,sq_err
4192,45,21,-8.0,24,1,38.5,66,1,1,0.667,18.67,17.67,1.00,0.500,14.50,22.38,-7.88,0.574,0.087,756.25
299,17,14,-3.5,3,0,36.0,31,0,0,0.000,6.40,28.80,-22.40,0.200,17.20,23.20,-6.00,0.000,0.160,25.00
2252,16,20,-3.0,4,0,50.0,36,0,1,0.714,29.00,19.00,10.00,0.714,28.14,19.43,8.71,0.921,0.895,196.00
1782,11,27,-2.5,16,0,37.0,38,1,1,0.500,15.50,18.25,-2.75,0.375,15.75,28.38,-12.63,0.297,0.034,1.00
1876,35,7,-7.5,28,0,32.0,42,1,0,0.357,19.00,22.57,-3.57,0.857,28.14,17.36,10.78,0.276,0.941,100.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
440,24,13,-12.0,11,0,40.0,37,0,1,0.714,23.71,16.29,7.42,0.357,19.57,24.79,-5.22,0.889,0.207,9.00
4428,36,31,-10.0,5,0,51.0,67,1,1,0.889,33.22,22.22,11.00,0.500,23.50,19.50,4.00,0.919,0.741,256.00
4556,17,19,-3.5,2,0,43.5,36,0,1,1.000,43.00,13.00,30.00,0.000,24.00,30.00,-6.00,0.999,0.208,56.25
1783,10,13,-2.5,3,1,40.5,23,0,0,0.000,20.75,30.50,-9.75,0.500,16.25,19.50,-3.25,0.093,0.273,306.25


### Mean Squared Error for the Bookmakers' on all these games

In [29]:
df['sq_err'].mean(skipna=True)

182.23118335772529

In [12]:
df.to_json('/Users/pranav/nfl_betting_system/final_data.json')
df.to_csv('/Users/pranav/nfl_betting_system/final_data.csv')